# 0. GPU check

* 이 코드는 Nvidia GPU를 사용하는 컴퓨터에서, train / test 데이터가 분리되어있는 csv 파일을 사용하는 것을 전제로 작성됨

In [1]:
import torch

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print("device_count: {}".format(device_count))
    for device_num in range(device_count):
        print("device {} capability {}".format(
            device_num,
            torch.cuda.get_device_capability(device_num)))
        print("device {} name {}".format(
            device_num, 
            torch.cuda.get_device_name(device_num)))
else:
    print("no cuda device")

device_count: 1
device 0 capability (8, 6)
device 0 name NVIDIA GeForce RTX 3080


In [2]:
if torch.cuda.is_available() :
    device = torch.device("cuda:0")
else : 
    device = torch.device("cpu")

In [3]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()
    
print_gpu_utilization()

GPU memory occupied: 424 MB.


* 모델 훈련과정에서 GPU 메모리 용량 초과 시, 개발서버 콘솔에서 직접 `nvidia-smi` 명령어 실행 후 메모리를 점유하고 있는 process의 PID를 찾아 `sudo kill -9 {pid}` 로 프로세스 종료해주면 됨

# 1. Import packages

In [4]:
## Need to check if packages are compatible
# !pip install accelerate nvidia-ml-py3
# !pip install datasets==2.4.0
# !pip install huggingface_hub==0.9.1
# !pip install transformers==4.22.1 # bf16, tf32 등 사용하려면 4.2 이상 필요
# !pip install pyarrow==9.0.0

* huggingface_hub와 transformers 간 호환가능한 버전 확인 필요
* 만약 성능 테스트를 위해 datasets api를 사용할거라면 datasets 역시 호환 가능 버전 확인해야 함
* 세 가지 dependencies를 사용한다는 가정 하에, pyarrow 라이브러리도 필요.

In [5]:
## Install libraries for optimizing hyperparameters

# !pip install ray optuna
# !pip install sigopt
# !pip install wandb

In [6]:
import transformers
import datasets
import huggingface_hub
import pyarrow

print(transformers.__version__)
print(datasets.__version__)
print(huggingface_hub.__version__)
print(pyarrow.__version__)

# 4.22.1
# 2.4.0
# 0.9.1
# 9.0.0

4.22.1
2.4.0
0.9.1
9.0.0


In [7]:
import os
import re
import math
import numpy as np
import pandas as pd

# 'You can use tf32' if you are acessing Ampere hardware
import torch
torch.backends.cuda.matmul.allow_tf32 = True

from datasets import load_dataset, load_metric, ClassLabel
from sklearn.utils.class_weight import compute_class_weight
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.examples.pbt_transformers.utils import (
    download_data,
    build_compute_metrics_fn,
)
from ray.tune.schedulers import PopulationBasedTraining
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)

# 2. Import Data

* xxx_train.csv, xxx_test.csv 파일은 아래 형식으로 전처리된 csv 파일이어야 함 (column name: `text`, `label`)


<table class="features-table">
  <tr>
    <th class="mdc-text-light-green-600", style="text-align:center">
    text
    </th>
    <th class="mdc-text-purple-600", style="text-align:center">
    label
    </th>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Ok lar... Joking wif u oni...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)
    </td>
    <td class="mdc-bg-purple-50">
      1
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      U dun say so early hor... U c already then say...
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
  <tr>
    <td class="mdc-bg-light-green-50" style="text-align:left">
      Nah I don't think he goes to usf, he lives around here though
    </td>
    <td class="mdc-bg-purple-50">
      0
    </td>
  </tr>
</table>

In [8]:
data_name = "IMDB" ## covid_articles / financial_news / IMDB / naver_movie_review / spam

dataset = load_dataset('csv', data_files={'train': f'../data_splited/{data_name}_train.csv',
                                          'test': f'../data_splited/{data_name}_test.csv'})
dataset

Using custom data configuration default-5e9b2acfce9f0b59
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 39999
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 9999
    })
})

# 3. Data Preprocessing

* load_dataset 함수로 불러온 데이터를 수정할 때는 수정 내용을 담은 함수를 만들고, 이를 map 함수로 각 원소에 적용함 ([링크](https://huggingface.co/docs/datasets/v1.4.0/processing.html#processing-data-row-by-row)에서 확인)

In [9]:
## remove specal characters

def remove_sp(example):
    example["text"]=re.sub(r'[^a-z|A-Z|0-9|ㄱ-ㅎ|ㅏ-ㅣ|가-힣| ]+', '', str(example["text"]))
    return example

dataset = dataset.map(remove_sp)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0c659aeae188f731.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-170d6a353647063a.arrow


In [10]:
## label encoding

labels = list(set(dataset["train"]["label"] + dataset["test"]["label"]))
num_labels = len(labels)

def encoding_label(example):
    str_to_int = ClassLabel(num_classes=num_labels, names=labels)
    example["label"]=str_to_int.str2int(example["label"])
    return example

if type(labels[0]) == str:
    dataset = dataset.map(encoding_label)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-82b60351b1fd1890.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-0fbc2fb5dafc34c1.arrow


# 4. Load PLM & Tokenizing

In [11]:
# model_name = "bert-base-multilingual-cased"
# model_name = "klue/bert-base"
# model_name = "klue/roberta-base"
model_name = "xlm-roberta-base"

In [12]:
# Download cache tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [13]:
def tokenize_function(examples):
    tokenized_batch = tokenizer(examples["text"], padding="max_length", truncation=True) # padding : ['longest', 'max_length', 'do_not_pad']
    return tokenized_batch

In [14]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-5d1b3f390e177513.arrow


  0%|          | 0/10 [00:00<?, ?ba/s]

In [15]:
# train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(0,math.floor(len(tokenized_datasets["train"])*0.7)))
# eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(math.floor(len(tokenized_datasets["train"])*0.7), len(tokenized_datasets["train"])))
# test_dataset = tokenized_datasets["test"]

In [16]:
# data for test
train_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(1000))
eval_dataset = tokenized_datasets["train"].shuffle(seed=1919).select(range(1000))
test_dataset = tokenized_datasets["test"]

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-67a5ed5b34b2fec1.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/csv/default-5e9b2acfce9f0b59/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a/cache-67a5ed5b34b2fec1.arrow


# 5. Check class weights

In [17]:
def class_weight(train_dataset) :
    
    train_labels = np.array(train_dataset["label"])
    class_weights = compute_class_weight(class_weight = 'balanced', classes = np.unique(train_labels), y = train_labels)
    
    weights = torch.tensor(class_weights, dtype = torch.float)
    
    return weights

In [18]:
weights = class_weight(train_dataset)
print(weights)

tensor([1.0225, 0.9785])


# 6. Modeling

In [19]:
task_data_dir = "test-model"
gpus_per_trial = 1
n_trials = 1
metric = load_metric("accuracy") # atasets.list_metrics() 

In [20]:
# Download model and features

config = AutoConfig.from_pretrained(
    model_name, 
    num_labels=num_labels
)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        config=config
        )

In [21]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions=np.argmax(logits, axis = -1)
    return metric.compute(predictions=predictions, references=labels)

```python
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1,   # batch size per device during training
    per_device_eval_batch_size=10,   # batch size for evaluation
    warmup_steps=1000,               # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,               # How often to print logs
    do_train=True,                   # Perform training
    do_eval=True,                    # Perform evaluation
    evaluation_strategy="epoch",     # evalute after each epoch
    gradient_accumulation_steps=64,  # total number of steps before back propagation
    fp16=True,                       # Use mixed precision
    fp16_opt_level="02",             # mixed precision mode
    run_name="ProBert-BFD-MS",       # experiment name
    seed=3                           # Seed for experiment reproducibility 3x3
)
```

In [22]:
training_args = TrainingArguments(
    output_dir=".",
    learning_rate=2e-5,
    do_train=True,
    do_eval=True,
    no_cuda=gpus_per_trial <= 0,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=20,  # config
    max_steps=-1,
    per_device_train_batch_size=6,  # config
    per_device_eval_batch_size=6,  # config
    warmup_steps=0,
    warmup_ratio=0.1,
    weight_decay=0.1,  # config
    logging_dir="./logs",
    skip_memory_metrics=True,
    report_to="none",
    fp16=True,
    # bf16=True,
    # tf32=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    seed=818
    )
    
# trainer = Trainer(
#     model_init=model_init,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     compute_metrics=compute_metrics,
#     )


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss
        weight = weights.to(device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    
trainer = CustomTrainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    )

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/f6d161e8f5f6f2ed433fb4023d6cb34146506b3f/pytorch_model.bin
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificat

In [24]:
# Hyperparameter tuning with ray tune

tune_config = {
    "per_device_train_batch_size": 6, 
    "per_device_eval_batch_size": 6,
#     "num_train_epochs": tune.choice([2, 3]),
    "max_steps": -1
}

# PopulationBasedTraining
# worker might copy the model parameters from a better performing worker or explore new hyperparameters by changing the current values randomly
# cf. ASHAScheduler
scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="eval_accuracy",
    mode="max",
    perturbation_interval=1,
    hyperparam_mutations={
        "weight_decay": tune.uniform(0.0, 0.3), # tune.uniform(1, 10) == np.random.uniform(1, 10)
#         "learning_rate": tune.uniform(1e-5, 5e-5),
        "per_device_train_batch_size": [6],
    },
)

reporter = CLIReporter(
    parameter_columns={
        "weight_decay": "w_decay",
        "learning_rate": "lr",
        "per_device_train_batch_size": "train_bs/gpu",
        "num_train_epochs": "num_epochs",
    },
    metric_columns=["eval_accuracy", "eval_loss", "epoch", "training_iteration"],
)

result = trainer.hyperparameter_search(
    hp_space = lambda _: tune_config,
    backend="ray",
    n_trials=n_trials,
    resources_per_trial={"cpu": 16, "gpu": gpus_per_trial},
    scheduler=scheduler,
    keep_checkpoints_num=1,
    checkpoint_score_attr="training_iteration",
    stop=None,
    progress_reporter=reporter,
    local_dir="./test-results",
    name="tune_transformer_pbt",
    log_to_file=True,
)

2022-10-11 03:55:58,262	INFO worker.py:1518 -- Started a local Ray instance.
/opt/conda/lib/python3.8/site-packages/ray/tune/tune.py:505: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py:642: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/ray-air/key-concepts.html#session

  warnings.warn(
2022-10-11 03:56:05,363	WARNING trial_runner.py:1575 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler

== Status ==
Current time: 2022-10-11 03:56:05 (running for 00:00:00.16)
Memory usage on this node: 8.8/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/15.02 GiB heap, 0.0/7.51 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 1/1 (1 RUNNING)
+------------------------+----------+--------------------+-----------+------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay | lr   |   train_bs/gpu | num_epochs   |
|------------------------+----------+--------------------+-----------+------+----------------+--------------|
| _objective_a15a2_00000 | RUNNING  | 172.17.0.3:2373378 |  0.258967 |      |              6 |              |
+------------------------+----------+--------------------+-----------+------+----------------+--------------+




(_objective pid=2373378) Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
(_objective pid=2373378) - This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=2373378) - This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=2373378) Some weights of XLMRobertaForSequenceClassification were not ini

== Status ==
Current time: 2022-10-11 03:56:12 (running for 00:00:07.41)
Memory usage on this node: 13.4/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/15.02 GiB heap, 0.0/7.51 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 1/1 (1 RUNNING)
+------------------------+----------+--------------------+-----------+------+----------------+--------------+
| Trial name             | status   | loc                |   w_decay | lr   |   train_bs/gpu | num_epochs   |
|------------------------+----------+--------------------+-----------+------+----------------+--------------|
| _objective_a15a2_00000 | RUNNING  | 172.17.0.3:2373378 |  0.258967 |      |              6 |              |
+------------------------+----------+--------------------+-----------+------+----------------+--------------+




  1%|          | 8/820 [00:04<07:45,  1.75it/s]


...

== Status ==
Current time: 2022-10-11 04:07:37 (running for 00:11:32.49)
Memory usage on this node: 13.4/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 16.0/20 CPUs, 1.0/1 GPUs, 0.0/15.02 GiB heap, 0.0/7.51 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspace/syc/BERT_classification_binary/test-results/tune_transformer_pbt
Number of trials: 1/1 (1 RUNNING)
+------------------------+----------+--------------------+-----------+------+----------------+--------------+-----------------+-------------+---------+----------------------+
| Trial name             | status   | loc                |   w_decay | lr   |   train_bs/gpu | num_epochs   |   eval_accuracy |   eval_loss |   epoch |   training_iteration |
|------------------------+----------+--------------------+-----------+------+----------------+--------------+-----------------+-------------+---------+----------------------|
| _objective_a15a2_00000 | RUNNING  | 172.17.0.3:2373378 |  0.258967

(_objective pid=2373378) 
 87%|████████▋ | 145/167 [00:04<00:00, 30.99it/s]
(_objective pid=2373378) 
 89%|████████▉ | 149/167 [00:04<00:00, 31.06it/s]
(_objective pid=2373378) 
 92%|█████████▏| 153/167 [00:04<00:00, 31.10it/s]
(_objective pid=2373378) 
 94%|█████████▍| 157/167 [00:05<00:00, 31.14it/s]
(_objective pid=2373378) 
 96%|█████████▋| 161/167 [00:05<00:00, 31.14it/s]
(_objective pid=2373378) 
 99%|█████████▉| 165/167 [00:05<00:00, 31.18it/s]
                                                 
100%|██████████| 167/167 [00:05<00:00, 31.18it/s]
                                                 


(_objective pid=2373378) {'eval_loss': 0.002041089115664363, 'eval_accuracy': 0.999, 'eval_runtime': 5.3572, 'eval_samples_per_second': 186.664, 'eval_steps_per_second': 31.173, 'epoch': 19.98}
Result for _objective_a15a2_00000:
  date: 2022-10-11_04-07-41
  done: false
  epoch: 19.98
  eval_accuracy: 0.999
  eval_loss: 0.002041089115664363
  eval_runtime: 5.3572
  eval_samples_per_second: 186.664
  eval_steps_per_second: 31.173
  experiment_id: 2db33cb4467541ffb8badd9bbe1fdeb1
  hostname: 3481a8a2ae33
  iterations_since_restore: 20
  node_ip: 172.17.0.3
  objective: 0.999
  pid: 2373378
  should_checkpoint: true
  time_since_restore: 694.060560464859
  time_this_iter_s: 34.205313205718994
  time_total_s: 694.060560464859
  timestamp: 1665461261
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: a15a2_00000
  warmup_time: 0.003078460693359375
  


100%|██████████| 820/820 [11:32<00:00,  1.18it/s]
2022-10-11 04:07:45,793	INFO tune.py:758 -- Total run time: 700.55 seconds (700.32 seconds for the tuning loop).


Result for _objective_a15a2_00000:
  date: 2022-10-11_04-07-41
  done: true
  epoch: 19.98
  eval_accuracy: 0.999
  eval_loss: 0.002041089115664363
  eval_runtime: 5.3572
  eval_samples_per_second: 186.664
  eval_steps_per_second: 31.173
  experiment_id: 2db33cb4467541ffb8badd9bbe1fdeb1
  experiment_tag: '0'
  hostname: 3481a8a2ae33
  iterations_since_restore: 20
  node_ip: 172.17.0.3
  objective: 0.999
  pid: 2373378
  should_checkpoint: true
  time_since_restore: 694.060560464859
  time_this_iter_s: 34.205313205718994
  time_total_s: 694.060560464859
  timestamp: 1665461261
  timesteps_since_restore: 0
  training_iteration: 20
  trial_id: a15a2_00000
  warmup_time: 0.003078460693359375
  
== Status ==
Current time: 2022-10-11 04:07:45 (running for 00:11:40.32)
Memory usage on this node: 13.4/31.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/20 CPUs, 0/1 GPUs, 0.0/15.02 GiB heap, 0.0/7.51 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /workspa

In [25]:
result

BestRun(run_id='a15a2_00000', objective=0.999, hyperparameters={'per_device_train_batch_size': 6, 'per_device_eval_batch_size': 6, 'max_steps': -1, 'weight_decay': 0.2589669844681074})

# Hyperparameters 반영 내용

scheduler는 ASHAScheduler 대신 PBT scheduler를 사용했고, metric, mode, max_t는 동일하게 적용  
AdamW Optimizer는 PBT scheduler에 적용할 수 없어, 관련 hyperparameters 제외하고 적용

num_samples = 1 -> n_trials 변수에 반영  
max_num_epochs = 20 -> 동일하게 반영  
seed = 818 -> 동일하게 반영  
model_type = modeltype -> model_name = "xlm-roberta-base"  
num_labels =  2 -> 동일하게 반영  
bias_correction =  True -> AdamW의 hyperparameters  
batch_size =  8 -> 동일하게 반영  
eps =  1e-8 -> AdamW의 hyperparameters  
warmup =  0.1 -> 동일하게 반영  
beta =  (0.9, 0.999) -> AdamW의 hyperparameters  
lr =  2e-5 -> AdamW의 hyperparameters이지만, TrainingArguments의 hyperparameters에 반영

# Reference

https://bo-10000.tistory.com/154
https://huggingface.co/blog/ray-tune  
https://docs.ray.io/en/latest/tune/examples/pbt_transformers.html
https://wood-b.github.io/post/a-novices-guide-to-hyperparameter-optimization-at-scale/#schedulers-vs-search-algorithms
https://docs.ray.io/en/latest/tune/api_docs/search_space.html
https://docs.ray.io/en/latest/tune/tutorials/tune-advanced-tutorial.html
https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/
https://docs.ray.io/en/latest/tune/faq.html